# Training a Neural Network with PyTorch

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd

## Load data

In [ ]:
data = pd.read_csv('/content/sample_data/california_housing_train.csv')
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [ ]:
#Fill NaN values
data = data.fillna(0)
#Normalize values
data = (data-data.mean())/data.std()
#Separate features and targets
x_df = pd.DataFrame(data, columns=data.columns[:-1])
y_df = pd.DataFrame(data, columns=[data.columns[-1]])
print(x_df)
print(y_df)

#Save in tensors
x = torch.tensor(x_df.values, dtype=torch.float)
y = torch.tensor(y_df.values, dtype=torch.float)

print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

       longitude  latitude  ...  households  median_income
0       2.619288 -0.671500  ...   -0.075996      -1.252506
1       2.539494 -0.573248  ...   -0.099401      -1.081451
2       2.494610 -0.905436  ...   -0.999223      -1.170071
3       2.489623 -0.928830  ...   -0.715753      -0.362590
4       2.489623 -0.961581  ...   -0.622130      -1.026424
...          ...       ...  ...         ...            ...
16995  -2.342894  2.318197  ...   -0.343862      -0.799975
16996  -2.347881  2.369663  ...   -0.094200      -0.715705
16997  -2.362842  2.907715  ...   -0.117606      -0.446650
16998  -2.362842  2.889000  ...   -0.060392      -0.997758
16999  -2.387778  2.299482  ...   -0.601325      -0.455349

[17000 rows x 8 columns]
       median_house_value
0               -1.210522
1               -1.096713
2               -1.048430
3               -1.154480
4               -1.222593
...                   ...
16995           -0.826848
16996           -1.106197
16997           -0.894099
16998 

## Create a Neural Network

In [ ]:
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, H3)
        self.linear4 = nn.Linear(H3, D_out)
        
    def forward(self, x):
        h1 = self.linear1(x)
        h2 = self.linear2(h1)
        h3 = self.linear3(h2)
        out = self.linear4(h3)
        return out

In [ ]:
#Define layer sizes
D_in = x.shape[1]
print(D_in)
H1 = 128
H2 = 64
H3 = 32
D_out = 1

#Define Hyperparameters
learning_rate = 1e-4

#Initialize model, loss, optimizer
model = Net(D_in, H1, H2, H3, D_out)
loss_func = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#Initialize dataloader
dataset = torch.utils.data.TensorDataset(x, y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
for data in dataloader:
  print (data)
  break

8
[tensor([[ 0.7441, -0.8119,  0.0326, -0.6907, -0.7649, -0.4056, -0.7262,  0.3424],
        [ 0.6045, -0.7510,  0.4299,  1.1809,  2.4593,  1.8368,  2.2620, -1.2417],
        [-0.8617,  1.1111, -1.9536, -0.5742, -0.5158,  0.3088, -0.4999, -0.7906],
        [ 0.6344, -0.7230,  0.7477, -1.0384, -0.9713, -0.8578, -0.9394, -1.0245],
        [ 0.6394, -0.7464,  1.3038, -0.7801, -0.5941, -0.4152, -0.6845, -0.9518],
        [ 1.1281, -0.6809,  0.1121, -0.0737,  0.0797, -0.2244, -0.1644, -1.0526],
        [-1.7544,  1.2889, -0.0468,  0.2997,  0.3241, -0.4413, -0.3335,  0.0174],
        [ 0.8040, -0.8820, -1.0002,  0.0570,  0.1437, -0.2157,  0.1607, -0.0877],
        [ 0.7890, -0.8820,  0.0326,  0.3667,  0.2268,  0.7208,  0.3089,  0.4490],
        [-1.1360,  0.7649, -1.8741, -0.3833, -0.4731, -0.5267, -0.4219,  0.6118],
        [-1.2856,  0.9707, -0.9207, -0.5416, -0.5870, -0.6400, -0.4453,  0.3681],
        [ 0.7491, -0.8353, -0.6824,  1.3759,  1.6408,  2.1008,  1.8043,  0.0894],
        [-1.3

In [ ]:
model

Net(
  (linear1): Linear(in_features=8, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=1, bias=True)
)

## Train Network

In [ ]:
for epoch in range(100):
  batch_losses = []

  for x_batch, y_batch in dataloader:
    y_pred = model(x_batch)
    
    loss = loss_func(y_pred, y_batch)
    batch_losses.append(loss.item())
    
    #Delete previously stored gradients
    optimizer.zero_grad()
    #Perform backpropagation starting from the loss calculated in this epoch
    loss.backward()
    #Update model's weights based on the gradients calculated during backprop
    optimizer.step()
  
  print(f"Epoch {epoch:3}: Loss = {sum(batch_losses)/len(dataloader):.5f}")
    

Epoch   0: Loss = 23.07366
Epoch   1: Loss = 23.05640
Epoch   2: Loss = 23.04923
Epoch   3: Loss = 23.08227
Epoch   4: Loss = 23.08470
Epoch   5: Loss = 23.03137
Epoch   6: Loss = 23.04910
Epoch   7: Loss = 23.06604
Epoch   8: Loss = 23.04250
Epoch   9: Loss = 23.03983
Epoch  10: Loss = 23.04872
Epoch  11: Loss = 23.04682
Epoch  12: Loss = 23.04315
Epoch  13: Loss = 23.03031
Epoch  14: Loss = 23.04879
Epoch  15: Loss = 23.04778
Epoch  16: Loss = 23.01762
Epoch  17: Loss = 23.04221
Epoch  18: Loss = 23.05282
Epoch  19: Loss = 23.02865
Epoch  20: Loss = 23.03847
Epoch  21: Loss = 22.99498
Epoch  22: Loss = 23.04007
Epoch  23: Loss = 23.05867
Epoch  24: Loss = 22.97663
Epoch  25: Loss = 23.05040
Epoch  26: Loss = 23.05760
Epoch  27: Loss = 23.04377
Epoch  28: Loss = 23.03008
Epoch  29: Loss = 23.04940
Epoch  30: Loss = 23.02832
Epoch  31: Loss = 23.04114
Epoch  32: Loss = 23.06131
Epoch  33: Loss = 23.02411
Epoch  34: Loss = 23.04587
Epoch  35: Loss = 23.04067
Epoch  36: Loss = 23.04783
E